# Homework week  2

In [ ]:
import pandas as pd

import numpy as np

from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  roc_auc_score

## Task 1

In [ ]:
data = pd.read_csv('./mcdonalds.csv')

In [ ]:
data.head()

In [ ]:
data.shape

## Task 2

In [ ]:
data_reduced = data[pd.notnull(data['policies_violated'])]

In [ ]:
data_reduced.shape

In [ ]:
data_reduced.head()

## Task 3

In [ ]:
data_reduced["rude"] = data_reduced["policies_violated"].str.contains("RudeService").apply(lambda i: int(i))

In [ ]:
data_reduced.head()

## Task 4

In [ ]:
X = data_reduced['review']
y = data_reduced['rude']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
cv = CountVectorizer()

In [ ]:
X_train_dtm = cv.fit_transform(X_train)

In [ ]:
X_test_dtm = cv.transform(X_test)

In [ ]:
print(X_train_dtm.shape[1])

## Task 5

In [ ]:
nb = MultinomialNB()

In [ ]:
nb.fit(X_train_dtm, y_train)

In [ ]:
pred_proba = nb.predict_proba(X_test_dtm)[:, 1]

In [ ]:
roc_auc_score(y_test, pred_proba)

In [ ]:
logreg = LogisticRegression()

In [ ]:
logreg.fit(X_train_dtm, y_train)

In [ ]:
pred_proba_lr = logreg.predict_proba(X_test_dtm)[:, 1]

In [ ]:
roc_auc_score(y_test, pred_proba_lr)

**NB is better**

## Task 6

In [ ]:
def tokenize_test(vect):
    
    X_train_dtm = vect.fit_transform(X_train)
    X_test_dtm = vect.transform(X_test)
    
    print(X_train_dtm.shape[1])
    
    nb = MultinomialNB()
    nb.fit(X_train_dtm, y_train)

    y_pred_proba = nb.predict_proba(X_test_dtm)[:,1]
    
    print("AUC: ", roc_auc_score(y_test, y_pred_proba))
    

In [ ]:
tokenize_test(CountVectorizer(ngram_range=(1, 2)))

In [ ]:
tokenize_test(CountVectorizer(ngram_range=(1, 2), max_df=0.5))

In [ ]:
tokenize_test(CountVectorizer(ngram_range=(1, 2), max_df=0.8))

In [ ]:
tokenize_test(CountVectorizer(max_df=0.8))

In [ ]:
tokenize_test(CountVectorizer(max_df=0.5))

In [ ]:
tokenize_test(CountVectorizer(max_df=0.8, min_df=2))

In [ ]:
tokenize_test(CountVectorizer(lowercase=False))

In [ ]:
tokenize_test(CountVectorizer(token_pattern='(?u)\\b\\w\\w\\w+\\b'))

In [ ]:
tokenize_test(CountVectorizer(token_pattern='(?u)\\b\\w\\w\\w+\\b', max_df=0.5))

**This is best result**

## Task 7

In [ ]:
data_reduced["review_city"] = data_reduced["review"].str.cat(data_reduced["city"], sep=" ", na_rep="na")

In [ ]:
X = data_reduced["review_city"]
y = data_reduced["rude"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
tokenize_test(CountVectorizer(token_pattern='(?u)\\b\\w\\w\\w+\\b', max_df=0.5))

In [ ]:
tokenize_test(CountVectorizer())

**I don't see any AUC significant change**

## Task 8

In [ ]:
data_reduced["confidence_list"] = data_reduced["policies_violated:confidence"].str.split("\n")

In [ ]:
def calc_mean(vals):
    print(vals)

In [ ]:
data_reduced["confidence_mean"] = data_reduced["confidence_list"].apply(lambda vals: np.mean([float(val) for val in vals]))

In [ ]:
data_with_confidence = data_reduced[data_reduced["confidence_mean"] == 1]

In [ ]:
data_with_confidence.shape

In [ ]:
data_with_confidence.head(50)

In [ ]:
X = data_with_confidence["review"]
y = data_with_confidence["rude"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [ ]:
tokenize_test(CountVectorizer())

In [ ]:
tokenize_test(CountVectorizer(token_pattern='(?u)\\b\\w\\w\\w+\\b', max_df=0.5))

In [ ]:
tokenize_test(CountVectorizer(max_df=0.5))

In [ ]:
tokenize_test(CountVectorizer(max_df=0.8))

In [ ]:
tokenize_test(CountVectorizer(min_df=2, max_df=0.5))

**Reducing training set to highest-quality data increased AUC but more tunning didn't**

## Task 9

In [ ]:
X = data_with_confidence["review"]
y = data_with_confidence["rude"]

In [ ]:
vect = CountVectorizer(min_df=2, max_df=0.5)

In [ ]:
X_dtm = vect.fit_transform(X)

In [ ]:
new_comments = pd.read_csv("./mcdonalds_new.csv")

In [ ]:
new_comments

In [ ]:
new_comments_dtm = vect.transform(new_comments["review"])

In [ ]:
new_comments_dtm.shape

In [ ]:
nb = MultinomialNB()
nb.fit(X_dtm, y)

In [ ]:
new_pred_prob = nb.predict_proba(new_comments_dtm)

In [ ]:
df = pd.DataFrame({"review": new_comments["review"], "rude probaility": new_pred_prob[:, 1]})

In [ ]:
pd.set_option('display.max_colwidth', -1)

In [ ]:
df

**Hmm, 2/10 are incorect so we have about 80% accuracy (like in tests).**